In [2]:
import requests
import pickle
from bs4 import BeautifulSoup
import lxml
import random
import pandas as pd
import time
random.seed = 456

In [2]:
session = requests.Session()

In [3]:
session.headers

{'User-Agent': 'python-requests/2.25.1', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}

In [4]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; en-US; rv:76.0) Gecko/20100101 Firefox/76.0'}

In [5]:
session.headers.update(headers)

In [6]:
session.headers

{'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; en-US; rv:76.0) Gecko/20100101 Firefox/76.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}

In [7]:
params_res_hw = {'fillQuickSearch':'false',
              'target':'advanced',
              'expand':'dl',
              'field1':'AllField',
              'text1':"Hardware AND Architecture",
              'Ppub':"[20210315 TO 20220315]",
              'pageSize':50,
              'startPage':0,
              'rel':'nofollow',
              'ContentItemType':'research-article'}

In [8]:
params_res_se = {'fillQuickSearch':'false',
              'target':'advanced',
              'expand':'dl',
              'field1':'AllField',
              'text1':"Software AND Engineering",
              'Ppub':"[20210315 TO 20220315]",
              'pageSize':50,
              'startPage':0,
              'rel':'nofollow',
              'ContentItemType':'research-article'}

In [9]:
params_res_fl = {'fillQuickSearch':'false',
              'target':'advanced',
              'expand':'dl',
              'field1':'AllField',
              'text1':"Fuzzy AND Logic",
              'Ppub':"[20210315 TO 20220315]",
              'pageSize':50,
              'startPage':0,
              'rel':'nofollow',
              'ContentItemType':'research-article'}

In [10]:
params_res_sm = {'fillQuickSearch':'false',
              'target':'advanced',
              'expand':'dl',
              'field1':'AllField',
              'text1':"Systems AND Control",
              'Ppub':"[20210315 TO 20220315]",
              'pageSize':50,
              'startPage':0,
              'rel':'nofollow',
              'ContentItemType':'research-article'}

In [11]:
params_res_db = {'fillQuickSearch':'false',
              'target':'advanced',
              'expand':'dl',
              'field1':'AllField',
              'text1':"Databases",
              'Ppub':"[20210315 TO 20220315]",
              'pageSize':50,
              'startPage':0,
              'rel':'nofollow',
              'ContentItemType':'research-article'}

In [12]:
params_res_cs = {'fillQuickSearch':'false',
              'target':'advanced',
              'expand':'dl',
              'field1':'AllField',
              'text1':"Computer AND Security",
              'Ppub':"[20210315 TO 20220315]",
              'pageSize':50,
              'startPage':0,
              'rel':'nofollow',
              'ContentItemType':'research-article'}

## Результаты

In [86]:
results = []

In [87]:
params_res = params_res_fl

In [88]:
r = session.get('https://dl.acm.org/action/doSearch',params=params_res)

In [89]:
r.status_code

200

In [90]:
r.url

'https://dl.acm.org/action/doSearch?fillQuickSearch=false&target=advanced&expand=dl&field1=AllField&text1=Fuzzy+AND+Logic&Ppub=%5B20210315+TO+20220315%5D&pageSize=50&startPage=0&rel=nofollow&ContentItemType=research-article'

In [68]:
def parse_page_result(params,page):
    params['startPage'] = page
    r = session.get('https://dl.acm.org/action/doSearch',params=params)
    soup = BeautifulSoup(r.text,'lxml')
    articles_r = soup.find_all('span', class_='hlFld-Title')
    titles = []
    for article in articles_r:
        titles.append(article.text)
    links = []
    for article in articles_r:
        links.append('https://dl.acm.org'+article.findChild("a")['href'])
    return titles, links

In [69]:
def parse_info_article(url):
    r = session.get(url)
    soup = BeautifulSoup(r.text,'lxml')
    authors = []
    for author in soup.find_all('span', class_='loa__author-name'):
        authors.append(author.text)
    meta_r = soup.find('div', class_='issue-item__detail')
    source = meta_r.a['title']
    date = meta_r.find('span','epub-section__date').text.strip()
    if len(date)>15:
        date = date[12:]
    try:
        abstract_r = soup.find('div', class_='abstractSection abstractInFull').find('div', class_='abstractSection abstractInFull')
        if abstract_r is None:
            abstract_r = soup.find('div', class_='abstractSection abstractInFull')
            if abstract_r is None:
                abstract = ''
            else:
                abstract = abstract_r.text
        else:
            abstract = abstract_r.text
    except AttributeError:
        abstract = ''
    return authors, date, abstract, source

In [70]:
#pages = []
#for i in range(30):
    #if len(pages) == 0:
        #pages.append(random.randint(1,2))
    #else:
        #pages.append(pages[i-1]+random.randint(1,5))

In [71]:
pages = range(8)

In [72]:
count_results = 0
for page in pages:
    result_info = parse_page_result(params_res,page)
    count_results_page = len(result_info[0])
    for i in range(count_results_page):
        results.append({'title':'','link':'','authors':'','date':'','abstract':'', 'source':''})
    for i,j in zip(range(count_results,count_results+count_results_page),range(count_results_page)):
        results[i].update(title=result_info[0][j], link=result_info[1][j])
    count_results += count_results_page
    time.sleep(10)

In [73]:
len(results)

400

In [74]:
#with open('pre_results_cs.pkl','wb') as pr:
    #pickle.dump(results,pr)

In [23]:
#with open('pre_results_hw.pkl','rb') as pr:
    #results = pickle.load(pr)

In [75]:
kol = 0
resp = []
for result in results:
    print(kol)
    result['authors'], result['date'], result['abstract'], result['source'] =  parse_info_article(result['link'])
    #time.sleep(0.5)
    kol+=1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [76]:
kol

400

In [97]:
results[50]

{'title': 'Research and Application of Hail Recognition Algorithm Based on X-Band Dual-Polarization Radar',
 'link': 'https://dl.acm.org/doi/10.1145/3474198.3478258',
 'authors': ['Hailong Wang', 'Jianxin He', 'Zhao Shi'],
 'date': 'May 2021',
 'abstract': 'Extreme weather such as hail can cause great losses to human production and life, and timely and accurate identification of hail can reduce the impact of such extreme weather. Polarization variables can be used to retrieve hydrometeor size, shape and precipitation type. This article mainly studies the hail recognition algorithm based on X-band dual-polarization radar. The algorithm uses the following parameters as input: the horizontal reflectivity Z, the differential reflectivity ZDR, the cross-correlation coefficient ρhv, the specific differential phase KDP, the standard deviation of horizontal reflectivity SD(Z), and the standard deviation of differential phase SD(ΦDP). First, use the hydrometeor classification algorithm to class

In [82]:
#with open('after_results_cs.pkl','wb') as ar:
    #pickle.dump(results,ar)

In [3]:
#with open('after_results_fl.pkl','rb') as ar:
    #results = pickle.load(ar)

In [79]:
for result in results: # без \n
    result['abstract']=result['abstract'][1:-1]

In [80]:
for result in results: # без latex формул
    result['abstract']=result['abstract'].replace('(<inline-formula> <tex-math notation="LaTeX">$d$ </tex-math></inline-formula>)','')

In [ ]:
# results[1438] -- in abstract (<inline-formula> <tex-math notation="LaTeX">$d$ </tex-math></inline-formula>) (old)
# results[1448] -- abstract: '\nAs .... states.\n' (old)

In [4]:
data = pd.DataFrame(results)

In [5]:
data

,title,link,authors,date,abstract,source
0,Fuzzy Logic in Surveillance Big Video Data Ana...,https://dl.acm.org/doi/10.1145/3444693,"[Khan Muhammad, Mohammad S. Obaidat, Tanveer H...",April 2022,CTV cameras installed for continuous surveilla...,ACM Computing Surveys
1,Robot Arm Control Using Fuzzy Logic Design Int...,https://dl.acm.org/doi/10.1145/3492547.3492620,"[Tawfiq Hussein Elmenfy, Mona Mohamed]",October 2021,Abstract— The main contribution of this propos...,ICEMIS'21: The 7th International Conference on...
2,Evaluation of Water Filtration System Through ...,https://dl.acm.org/doi/10.1145/3470716.3470734,"[Analyn Yumang, Lois Anne Borrega, Almer Dave ...",April 2021,Physicochemical parameters of water that have ...,ICIEI 2021: 2021 The 6th International Confere...
3,An Implementation of Fuzzy Logic Technique to ...,https://dl.acm.org/doi/10.1145/3492547.3492639,"[Abdelaziz Mahmod Badi, Sulayman Hussian Ameit...",October 2021,This work aims to study the effect of wax depo...,ICEMIS'21: The 7th International Conference on...
4,Research on Automatic Location of Seed Points ...,https://dl.acm.org/doi/10.1145/3510858.3511380,"[Tianyu Zhao, Hang Dai]",December 2021,Breast cancer is the second leading cause of c...,ICASIT 2021: 2021 International Conference on ...
...,...,...,...,...,...,...
395,Constantine: Automatic Side-Channel Resistance...,https://dl.acm.org/doi/10.1145/3460120.3484583,"[Pietro Borrello, Daniele Cono D'Elia, Leonard...",November 2021,In the era of microarchitectural side channels...,CCS '21: Proceedings of the 2021 ACM SIGSAC Co...
396,A Practical Approach for Dynamic Taint Trackin...,https://dl.acm.org/doi/10.1145/3485464,"[Katherine Hough, Jonathan Bell]",April 2022,"ynamic taint tracking, a technique that traces...",ACM Transactions on Software Engineering and M...
397,Intrusion Detection Based on the Game Theory,https://dl.acm.org/doi/10.1145/3474198.3478267,"[Yuqing Cheng, HaiYan Fu, Xuechao Sun]",May 2021,"With the development of computer technology, m...",ICFEICT 2021: International Conference on Fron...
398,Early warning analysis and empirical research ...,https://dl.acm.org/doi/10.1145/3508546.3508571,"[Yuhang Lin, Jiwei Zhu, Nan Lu]",December 2021,"In recent years, with the upgrading of macroec...",ACAI'21: 2021 4th International Conference on ...


In [25]:
results[334]

{'title': 'Reticle: a virtual machine for programming modern FPGAs',
 'link': 'https://dl.acm.org/doi/10.1145/3453483.3454075',
 'authors': ['Luis Vega',
  'Joseph McMahan',
  'Adrian Sampson',
  'Dan Grossman',
  'Luis Ceze'],
 'date': 'June 2021',
 'abstract': 'Modern field-programmable gate arrays (FPGAs) have recently powered high-profile efficiency gains in systems from datacenters to embedded devices by offering ensembles of heterogeneous, reconfigurable hardware units. Programming stacks for FPGAs, however, are stuck in the past—they are based on traditional hardware languages, which were appropriate when FPGAs were simple, homogeneous fabrics of basic programmable primitives. We describe Reticle, a new low-level abstraction for FPGA programming that, unlike existing languages, explicitly represents the special-purpose units available on a particular FPGA device. Reticle has two levels: a portable intermediate language and a target-specific assembly language. We show how to use 